<h1 style="color:blue;">Co-Occurance Based Recommendation Model</h1>

## Loading Shopping Cart and Filtering

In [2]:
# Block 1: Loading required libraries
# Gerekli kütüphaneler yükleniyor
import pandas as pd

In [ ]:
# Block 2: Load the ShoppingCart data with the correct delimiter
# ShoppingCart verisini doğru ayırıcı ile yükleyin
shopping_cart_file = "ShoppingCart.csv"
df_shopping_cart = pd.read_csv(shopping_cart_file, delimiter=';')

# Display the exact column names in the dataset to verify them
# Veri kümesindeki tam sütun adlarını doğrulamak için görüntüleyin
print("Column names:", df_shopping_cart.columns)

In [4]:
# Block 3: Prompt for user input on filter criteria
# Filtre kriterleri için kullanıcı girdisi iste
# Ask the user to specify the minimum transaction count per customer
# Kullanıcıdan her müşteri için minimum işlem sayısını belirlemesini iste
min_transactions = int(input("Enter the minimum transaction count per customer (enter 3 for default): "))  

# Ask the user to specify the minimum item count per shopping cart
# Kullanıcıdan her alışveriş sepeti için minimum ürün sayısını belirlemesini iste
min_cart_items = int(input("Enter the minimum item count per shopping cart (enter 2 for default): ")) 

In [ ]:
# Block 4: Filter data based on the user-defined criteria
# Blok 4: Kullanıcı tanımlı kriterlere göre veriyi filtrele

# Step 1: Filter finalized orders by excluding rows with NaN in OrderId and only including Active == False
# Adım 1: OrderId sütununda NaN olanları hariç tutarak ve yalnızca Active == False olanları dahil ederek tamamlanmış siparişleri filtreleyin
df_orders = df_shopping_cart[(df_shopping_cart['OrderId (ShoppingCart)'].notna()) & 
                             (df_shopping_cart['Active (ShoppingCart)'] == False)]

# Step 2: Exclude blacklisted customers
# Adım 2: Kara listeye alınmış müşterileri hariç tut
blacklisted_customers = [9620, 10246, 9063, 10469, 2681, 2825, 522, 10294, 9945, 8414]
df_orders = df_orders[~df_orders['CustomerId (ShoppingCart)'].isin(blacklisted_customers)]

# Step 3: Filter users (UserId) with transactions meeting the minimum transaction count
# Adım 3: Minimum işlem sayısını karşılayan işlemleri olan kullanıcıları (UserId) filtrele
user_transaction_counts = df_orders.groupby('UserId (ShoppingCart)')['OrderId (ShoppingCart)'].nunique()
eligible_users = user_transaction_counts[user_transaction_counts >= min_transactions].index
df_filtered = df_orders[df_orders['UserId (ShoppingCart)'].isin(eligible_users)]

# Step 4: Filter shopping carts that have at least the minimum item count specified by the user
# Adım 4: Kullanıcı tarafından belirtilen minimum ürün sayısını içeren alışveriş sepetlerini filtrele
order_item_counts = df_filtered.groupby('OrderId (ShoppingCart)')['ProductId (ShoppingCart)'].nunique()
eligible_orders = order_item_counts[order_item_counts >= min_cart_items].index
df_filtered = df_filtered[df_filtered['OrderId (ShoppingCart)'].isin(eligible_orders)]

# Display the number of rows after filtering
# Filtreleme işleminden sonraki satır sayısını görüntüle
print(f"Number of rows after filtering: {len(df_filtered)}")


In [ ]:
# Block 5: Calculate and display statistics on the filtered data
# Filtrelenmiş veriyle ilgili istatistikleri hesaplayın ve görüntüleyin

# Step 1: Calculate the total number of eligible customers in the filtered dataset
# Adım 1: Filtrelenmiş veri kümesindeki uygun müşterilerin toplam sayısını hesapla
total_customers = df_filtered['CustomerId (ShoppingCart)'].nunique()

# Step 2: Calculate the total number of eligible orders in the filtered dataset
# Adım 2: Filtrelenmiş veri kümesindeki uygun siparişlerin toplam sayısını hesapla
total_orders = df_filtered['OrderId (ShoppingCart)'].nunique()

# Step 3: Display the filtered data statistics
# Adım 3: Filtrelenmiş veri istatistiklerini görüntüleyin
print("Filtered Data Statistics:")
print(f"Total Eligible Customers: {total_customers}")  # Toplam Uygun Müşteri Sayısı
print(f"Total Eligible Orders: {total_orders}")        # Toplam Uygun Sipariş Sayısı


In [ ]:
# Block 6: Keep only the necessary columns to optimize processing speed
# Blok 6: İşlem hızını optimize etmek için yalnızca gerekli sütunları saklayın

# Selecting relevant columns for recommendation model
# Tavsiye modeli için ilgili sütunları seçme
df_filtered = df_filtered[['UserId (ShoppingCart)', 'CustomerId (ShoppingCart)', 
                           'OrderId (ShoppingCart)', 'ProductId (ShoppingCart)', 
                           'Quantity (ShoppingCart)']]

# Display the first few rows to verify the selected columns
# Seçilen sütunları doğrulamak için ilk birkaç satırı görüntüleyin
df_filtered.head()


## Loading Customer and Product Data

In [8]:
import pandas as pd

# Load customer and product data files
# Müşteri ve ürün veri dosyalarını yükle
customers_file = "FilteredCustomers.csv"
products_file = "Products.csv"

df_customers = pd.read_csv(customers_file, delimiter=';')
df_products = pd.read_csv(products_file, delimiter=';')

In [ ]:
df_customers.head()

In [ ]:
df_products.head()

In [11]:
# Block 2: Create Lookup Functions
# Eşleme Fonksiyonları Oluşturma

# Function to get customer code and title based on CustomerId
# CustomerId'ye göre müşteri kodunu ve ünvanını alma fonksiyonu

def get_customer_code(customer_id):
    customer_row = df_customers[df_customers['Id (Customers)'] == customer_id]
    if not customer_row.empty:
        customer_code = customer_row['Code (Customers)'].values[0]
        return customer_code
    else:
        return "Customer code not found"

# Function to get product model number based on ProductId
# ProductId'ye göre ürün model numarasını alma fonksiyonu
def get_product_model_number(product_id):
    model_number = df_products.loc[df_products['Id (Products)'] == product_id, 'ModelNumber (Products)']
    if not model_number.empty:
        return model_number.values[0]
    else:
        return "Product not found" 

# Function to get ProductId based on Product Model Number
# Ürün model numarasına göre ProductId alma fonksiyonu
def get_product_id_by_model_number(model_number):
    product_row = df_products[df_products['ModelNumber (Products)'] == model_number]
    if not product_row.empty:
        return product_row['Id (Products)'].values[0]
    else:
        return "ProductId not found"

# Function to get CustomerId based on Customer Code
# Müşteri koduna göre CustomerId alma fonksiyonu
def get_customer_id_by_code(customer_code):
    customer_row = df_customers[df_customers['Code (Customers)'] == customer_code]
    if not customer_row.empty:
        return customer_row['Id (Customers)'].values[0]
    else:
        return "CustomerId not found" 


In [ ]:
# Block 3: Test the Lookup Functions with Random Sample
# Blok 3: Eşleme Fonksiyonlarını Rastgele Örnek ile Test Etme

import random

## Randomly select a customer ID from the filtered data
## Filtrelenmiş veriden rastgele bir müşteri ID'si seçin
sample_customer_id = random.choice(df_filtered['CustomerId (ShoppingCart)'].unique())
customer_code = get_customer_code(sample_customer_id)
print(f"Customer Code for ID {sample_customer_id}: {customer_code}")

# Randomly select a product ID from the filtered data
# Filtrelenmiş veriden rastgele bir ürün ID'si seçin
sample_product_id = random.choice(df_filtered['ProductId (ShoppingCart)'].unique())
product_model_number = get_product_model_number(sample_product_id)
print(f"Product Model Number for ID {sample_product_id}: {product_model_number}")

# Test: Retrieve and print the ProductId for a specific product model number
# Test: Belirli bir ürün model numarası için ProductId'yi alın ve yazdırın
sample_model_number = "TRW GDB400"  # Replace with an actual model number from your dataset
product_id = get_product_id_by_model_number(sample_model_number)
print(f"ProductId for Model Number {sample_model_number}: {product_id}")

# Test: Retrieve and print the CustomerId for a specific customer code
# Test: Belirli bir müşteri kodu için CustomerId'yi alın ve yazdırın
sample_customer_code = "120.01.269"  # Replace with an actual customer code from your dataset
customer_id = get_customer_id_by_code(sample_customer_code)
print(f"CustomerId for Customer Code {sample_customer_code}: {customer_id}")

## DATA EXPLORATION

In [ ]:
# Block 1: Basic Statistics
# Temel İstatistikler

# Step 1: Count unique customers, orders, and products
# Adım 1: Benzersiz müşteri, sipariş ve ürün sayılarını hesapla
num_customers = df_filtered['CustomerId (ShoppingCart)'].nunique()
num_orders = df_filtered['OrderId (ShoppingCart)'].nunique()
num_products = df_filtered['ProductId (ShoppingCart)'].nunique()

# Display basic statistics
# Temel istatistikleri görüntüle
print("Basic Statistics:")
print(f"Unique Customers: {num_customers}")      
print(f"Unique Orders: {num_orders}")            
print(f"Unique Products: {num_products}")        


In [ ]:
# Block 2: Top Customers by Order Volume
# Sipariş Hacmine Göre En Çok Sipariş Veren Müşteriler

# Group by CustomerId and count the number of orders per customer
# CustomerId'ye göre gruplandır ve müşteri başına sipariş sayısını hesapla
top_customers = df_filtered.groupby('CustomerId (ShoppingCart)')['OrderId (ShoppingCart)'].nunique()
top_customers = top_customers.sort_values(ascending=False).head(100)

# Display top customers and their order counts with code and title
# En çok sipariş veren müşterileri, kodlarını ve ünvanlarını görüntüleyin
print("Top Customers by Order Volume:")
for customer_id, order_count in top_customers.items():
    customer_code = get_customer_code(customer_id)
    print(f"Customer ID: {customer_id}, Code: {customer_code}, Orders: {order_count}")


In [ ]:
# Block 3: Top Products by Popularity
# Popülerliğe Göre En Çok Sipariş Edilen Ürünler

# Group by ProductId and count distinct orders to find popular products
# ProductId'ye göre gruplandır ve benzersiz sipariş sayılarını hesaplayarak popüler ürünleri bul
top_products = df_filtered.groupby('ProductId (ShoppingCart)')['OrderId (ShoppingCart)'].nunique()
top_products = top_products.sort_values(ascending=False).head(100)

# Display top products and their order counts with model numbers
# En çok sipariş edilen ürünleri ve model numaralarını görüntüleyin
print("Top Products by Popularity:")
for product_id, order_count in top_products.items():
    product_model_number = get_product_model_number(product_id)
    print(f"Product ID: {product_id}, Model Number: {product_model_number}, Orders: {order_count}")


In [ ]:
# Block 4: Average Order Size
# Ortalama Sipariş Büyüklüğü

# Calculate the average number of unique products per order
# Sipariş başına ortalama benzersiz ürün sayısını hesapla
avg_products_per_order = df_filtered.groupby('OrderId (ShoppingCart)')['ProductId (ShoppingCart)'].nunique().mean()

# Display the average order size
# Ortalama sipariş büyüklüğünü görüntüleyin
print(f"Average Order Size (Unique Products per Order): {avg_products_per_order:.2f}")


## CREATING MATRIX

In [ ]:
print(df_filtered.columns)

In [ ]:
# Block: Create Item-Item Co-Occurrence Matrix
# Blok: Ürün-Ürün Birliktelik Matrisini Oluşturma

import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix

# Step 1: Extract unique products and create an index map
# Adım 1: Benzersiz ürünleri çıkarın ve bir indeks haritası oluşturun
product_ids = df_filtered['ProductId (ShoppingCart)'].drop_duplicates().reset_index(drop=True)
product_index_map = {pid: idx for idx, pid in enumerate(product_ids)}

# Step 2: Group by orders to get all products per order
# Adım 2: Siparişlere göre gruplandırın ve her sipariş için tüm ürünleri alın
orders = df_filtered.groupby('OrderId (ShoppingCart)')['ProductId (ShoppingCart)'].apply(list)

# Step 3: Build a co-occurrence dictionary
# Adım 3: Birliktelik sözlüğü oluşturun
from collections import defaultdict
co_occurrence_counts = defaultdict(int)

# For each order, generate all unique pairs of products and increment their co-occurrence
# Her sipariş için, tüm benzersiz ürün çiftlerini oluşturun ve birlikte görünme sayılarını artırın
for prod_list in orders:
    # Sort the product list to ensure consistency in pair generation
    # Ürün listesini sıralayın, böylece çift oluşturma tutarlı olsun
    prod_list = sorted(set(prod_list))
    for i in range(len(prod_list)):
        for j in range(i+1, len(prod_list)):
            p_i = product_index_map[prod_list[i]]
            p_j = product_index_map[prod_list[j]]
            co_occurrence_counts[(p_i, p_j)] += 1
            co_occurrence_counts[(p_j, p_i)] += 1  # Symmetric increment for (j, i)

# Step 4: Construct sparse matrix from co-occurrence dictionary
# Adım 4: Birliktelik sözlüğünden seyrek matris oluşturun
num_products = len(product_ids)
row_indices = []
col_indices = []
data_values = []

for (i, j), count in co_occurrence_counts.items():
    row_indices.append(i)
    col_indices.append(j)
    data_values.append(count)

co_occurrence_matrix = csr_matrix((data_values, (row_indices, col_indices)), shape=(num_products, num_products))

# Display basic info about the co-occurrence matrix
# Birliktelik matrisi hakkında temel bilgileri görüntüleyin
print("Item-Item Co-Occurrence Matrix Constructed:")
print(f"Shape: {co_occurrence_matrix.shape}")
print(f"Number of Non-Zero Entries: {co_occurrence_matrix.nnz}")


In [ ]:
print(co_occurrence_matrix)

In [ ]:
print(orders)

## RECOMMENDATION FUNCTION

In [44]:
# Block: Recommendation Function
# Blok: Öneri Fonksiyonu

def recommend_products_from_models(test_cart_model_numbers, top_n=5):
    """
    Recommends products based on the item-item co-occurrence matrix using product model numbers.
    Ürün model numaralarını kullanarak ürün-ürün birliktelik matrisine dayalı olarak ürün önerir.
    
    Parameters:
    - test_cart_model_numbers: List of product model numbers in the user's test cart
    - top_n: Number of recommendations to return (default: 5)
    """
    # Step 1: Convert product model numbers to ProductIds
    # Adım 1: Ürün model numaralarını ProductId'lere dönüştürün
    test_cart_product_ids = [get_product_id_by_model_number(model) for model in test_cart_model_numbers]
    test_cart_product_ids = [pid for pid in test_cart_product_ids if pid != "ProductId not found"]
    
    if not test_cart_product_ids:
        print("No valid products found in the test cart.")  # Test sepetinde geçerli ürün bulunamadı
        return []
    
    # Step 2: Map ProductIds to matrix indices
    # Adım 2: ProductId'leri matris indekslerine eşleyin
    product_index_map = {pid: idx for idx, pid in enumerate(df_filtered['ProductId (ShoppingCart)'].drop_duplicates().reset_index(drop=True))}
    index_product_map = {idx: pid for pid, idx in product_index_map.items()}  # Reverse mapping
    test_cart_indices = [product_index_map[pid] for pid in test_cart_product_ids if pid in product_index_map]
    
    if not test_cart_indices:
        print("No valid products found in the test cart indices.")  # Test sepetinde geçerli ürün indeksleri bulunamadı
        return []
    
    # Step 3: Aggregate co-purchase counts for products in the test cart
    # Adım 3: Test sepetindeki ürünler için birlikte satın alma sayılarını birleştirin
    co_purchase_scores = co_occurrence_matrix[test_cart_indices, :].sum(axis=0).A1  # Sum rows and convert to array
    
    # Step 4: Filter out products already in the test cart
    # Adım 4: Test sepetinde zaten bulunan ürünleri hariç tutun
    for idx in test_cart_indices:
        co_purchase_scores[idx] = 0  # Set co-purchase scores for test cart products to 0
    
    # Step 5: Rank products by co-purchase scores
    # Adım 5: Ürünleri birlikte satın alma puanlarına göre sıralayın
    top_indices = np.argsort(co_purchase_scores)[::-1][:top_n]  # Get top N indices in descending order
    
    # Map indices back to ProductIds
    # İndeksleri yeniden ProductId'lere eşleyin
    recommended_products = [index_product_map[idx] for idx in top_indices]
    
    # Return recommended products
    # Önerilen ürünleri döndürün
    return recommended_products


## CREATING TEST CART FOR RECOMMENDATIONS

In [47]:
# Block: Test Recommendation Function
# Blok: Öneri Fonksiyonunu Test Etme

def test_recommendation_function():
    """
    Tests the recommendation function by taking product model numbers from the user.
    Kullanıcıdan ürün model numaralarını alarak öneri fonksiyonunu test eder.
    """
    # Step 1: Get test shopping cart product model numbers from the user
    # Adım 1: Kullanıcıdan test alışveriş sepeti ürün model numaralarını alın
    test_cart_model_numbers = input("Enter test cart product model numbers (comma-separated): ").split(",") 
    test_cart_model_numbers = [model.strip() for model in test_cart_model_numbers]  # Remove extra spaces
    
    # Step 2: Generate recommendations using the recommendation function
    # Adım 2: Öneri fonksiyonunu kullanarak öneriler oluşturun
    recommended_products = recommend_products_from_models(test_cart_model_numbers)
    
    # Step 3: Display Recommendations
    # Adım 3: Önerileri Görüntüleyin
    if recommended_products:
        print("\nRecommended Products:")
        for pid in recommended_products:
            model_number = get_product_model_number(pid)
            print(f"Model Number: {model_number}")
    else:
        print("No recommendations available.")


In [ ]:
# Run the test function
test_recommendation_function()